In [47]:
import tweepy,re
from tweepy import Stream
#from tweepy.stream import StreamListener 
from tweepy import OAuthHandler
import json
from geotext import GeoText
from urllib2 import urlopen
from bs4 import BeautifulSoup
from nltk.tag import StanfordNERTagger
import urllib,urllib2
from lxml import html
import requests
import snowballstemmer
from nltk.corpus import stopwords
import ssl
from geopy.geocoders import Nominatim
from gensim.summarization import summarize
from collections import defaultdict
from textblob import TextBlob

In [16]:
def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''

    consumer_key = 'qiOaRLVPldazXmYmF3IaIQw4L'
    consumer_secret = 'FAzcCMF1UUyuNuSeddAA1nDJYPeXm6OhaCD084k1t3BZ0HleCY'
    access_token = '717220472623071233-oeDgXLyYdqT92Mi06aaAGV7EtSExfKS'
    access_secret = 'tVh6WAqVqJ5Pekb3skPON4OD46dyyBAIGOiWjPkZrtglC'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

In [17]:
tweets_data = []
api = load_api()
search_phrases = ['#EdPolicy','EdReform','#CommonCore','#CCSS','#CCChat','#ESSA']
maxTweets = 100

In [18]:
tweets_data = []
#f = open('tweets.json','a')
for search_phrase in search_phrases:
    for status in tweepy.Cursor(api.search,q=search_phrase,since="2017-11-10",until="2017-11-24",lang="en").items(maxTweets):
        tweets_data.append(status._json)
        #f.write('\n')
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [5]:
print len(tweets_data)

335


In [48]:
stanford_ner_dir = '/home/dell/stanford-ner-2015-04-20/'
eng_model_filename= stanford_ner_dir + 'classifiers/english.all.3class.distsim.crf.ser.gz'
my_path_to_jar= stanford_ner_dir + 'stanford-ner.jar'

st = StanfordNERTagger(model_filename=eng_model_filename, path_to_jar=my_path_to_jar)
enc = lambda x: x.encode('latin1', errors='ignore')
# regex for url and contact number
#url_re = "(http|ftp|https|www).://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
url_re = "^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$"
tel_re = "^[t][e][l][0-9-:()]+"
#list of all stopwords in english language
stopwords = set(stopwords.words('english'))
stemmer = snowballstemmer.stemmer('english')

In [36]:
def get_urltext(url):
    try:
        context = ssl._create_unverified_context()
        page = urllib2.urlopen(url,context = context).read()
        #r = requests.get(url)
    #except urllib2.HTTPError, e:
    except:
        return None
    soup = BeautifulSoup(page,"html.parser")
    content = "".join([p.get_text() for p in soup.find_all('p')])
    return content

In [37]:
def stopwords_removal(text):
    text = re.sub(r'[^A-Za-z0-9]', ' ', text)
    clean_text = text.split(' ')
    text = []
    for i in range(len(clean_text)):
        if clean_text[i] not in stopwords and len(clean_text[i])>0:
            text.append(clean_text[i])
    return ' '.join(text)

In [70]:
def get_location(text):
    text = text.title()
    places = GeoText(text)
    return places.cities

In [71]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [72]:
def get_place_name(coord):
    coords = re.split(',|\s+',coord)
    #print coords
    geolocator = Nominatim()
    try:
        location = geolocator.reverse("{}, {}".format(coords[1],coords[2]))
        return get_location(location.address)
    except:
        return None

In [73]:
def get_country_name(coord):
    coords = re.split(',|\s+',coord)
    geolocator = Nominatim()
    try:
        location = geolocator.reverse("{}, {}".format(coords[1],coords[2]))
        address = location.address
        address = address.title()
        places = GeoText(address)
        return places.countries[0]
    except:
        return None

In [74]:
def update_dict(location,text):
    global tweets_dict
    if location in tweets_dict.keys():
        tweets_dict[location].append(text)
    else:
        tweets_dict[location] = []
        tweets_dict[location].append(text)

In [77]:
count = 0
global tweets_dict 
tweets_dict = defaultdict(list)
for tweet in tweets_data:
    count += 1
    if len(tweet['entities']['urls'])>0:
        text = tweet['text']
        exp_url_text = get_urltext(tweet['entities']['urls'][0]['expanded_url'])
        if exp_url_text is not None and exp_url_text :
            text += exp_url_text
        loc = get_location(text)
        if len(loc)>0:
            location = most_common(loc)
            #print location
            update_dict(location,text)
        else:
            location = tweet['user']['location'].lower()
            if not re.search(url_re,location) and not re.search(tel_re,location) :
                if ':' in location:
                    loc = get_place_name(re.split(':',location)[1])
                    if loc is None:
                        loc = get_location(location)
                else :
                    loc = get_location(location)
                if len(loc) > 0:
                    location = loc[0]
                    update_dict(location,text)
                else:
                    #location = tweet['user']['location'].encode('utf-8')
                    if ':' in location:
                        point = re.split(':',location)[1]
                        country = get_country_name(point)
                        if country is not None and country:
                            location = country
                    update_dict(location,text)
    else:
        text = tweet['text']
        loc = get_location(text)
        if len(loc) > 0:
            location = most_common(loc)
            update_dict(location,text)
        else:
            location = tweet['user']['location'].lower()
            if not re.search(url_re,location) and not re.search(tel_re,location) :
                if ':' in location:
                    loc = get_place_name(re.split(':',location)[1])
                    if loc is  None:
                        loc = get_location(location)
                else :
                    loc = get_location(location)
                if len(loc) > 0:
                    location = loc[0]
                    update_dict(location,text)
                else:
                    #location = tweet['user']['location'].encode('utf-8')
                    if ':' in location:
                        point = re.split(':',location)[1]
                        country = get_country_name(point)
                        if country is not None and country :
                            location = country
                    update_dict(location,text)

In [29]:
print '\u2708\ufe0f'.decode('utf-8')

\u2708\ufe0f


In [76]:
for tweet in tweets_data:
    location = tweet['user']['location']
    if ':' in location:
        print 'location :' + location
        coord = location.split(':')[1]
        print get_place_name(coord)
        if get_place_name(coord) is None:
            print get_country_name(coord)

location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']
location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']
location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']
location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']
location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']
location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']
location :ÜT: 38.7973156,-77.1318254
[u'Rose Hill', u'Virginia']


In [78]:
#for key in tweets_dict.keys():
#    key = key.encode('utf-8')
print tweets_dict.keys()

[u'', u'Wilson', u'Perth', u'earth', u'maine', u'Texas', u'Medicine Hat', u'March', u'Tulsa', u'Hull', u'Bo', u'lives in the dead zone', u'united kingdom', u'El Paso', u'aloha as of late', u'australia', u'fort mitchell, ky', u'Of', u'Lexington', u'california, usa', u'alpena public schools, mi', u'in your head!', u'Alexandria', u'Liberal', u'east haddam, ct', u'Washington', u'massachusetts, usa', u'London', u'Deer Park', u'Seattle', u'Windsor', u'Sparks', u'auckland nz', u'Denver', u'Napa', u'Curitiba', u'Maryland', u'Vienna', u'canada', u'Iowa', u'Sheffield', u'Michigan', u'San Diego', u'newberry springs, ca', u'hawaii, usa', u'Los Angeles', u'east yorkshire ', u'Bangalore', u'rockford il', u'Cleveland', u'arizona state university', u'Hoboken', u'illinois, usa', u'Cambridge', u'south carolina, usa', u'western australia', u'obama coup hq, dc & russia', u'Rose Hill', u'Boston', u'Trumbull', u'el paso texas', u'Qom', u'new mexico', u'maine - the way life should be', u'Dallas', u'Asia', u'

In [23]:
#print tweets_dict.keys()
geolocator = Nominatim()
print get_place_name("38.7973156,-77.1318254")
location = geolocator.reverse("38.7973156,-77.1318254")
country = GeoText(location.address).countries
print country[0].encode('utf-8')

[u'Rose Hill', u'Virginia']
United States


In [79]:
positive_dicts = defaultdict(list)
negative_dicts = defaultdict(list)
neutral_dicts = defaultdict(list)
for location in tweets_dict.keys():
    for text in tweets_dict[location]:
        #print type(text)
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            if location in positive_dicts.keys():
                positive_dicts[location].append(text)
            else:
                positive_dicts[location] = []
                positive_dicts[location].append(text)
        elif analysis.sentiment.polarity < 0:
            if location in negative_dicts.keys():
                negative_dicts[location].append(text)
            else:
                negative_dicts[location] = []
                negative_dicts[location].append(text)
        else:
            if location in neutral_dicts.keys():
                neutral_dicts[location].append(text)
            else:
                neutral_dicts[location] = []
                neutral_dicts[location].append(text)

In [80]:
print "positive" + ":" + str(len(positive_dicts.keys()))
print "negative" + ":" + str(len(negative_dicts.keys()))
print "neutral" + ":" + str(len(neutral_dicts.keys()))

positive:73
negative:22
neutral:38
